<a href="https://colab.research.google.com/github/AlphAxe/Deep-NN-for-MNIST-Dataset/blob/master/Deep_NN_for_MNIST_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import important packages**

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


**Loading Data**

In [ ]:
mnist_ds, mnist_info = tfds.load(name = 'mnist', with_info=True, as_supervised= True)


**Data Preprocessing**

In [ ]:
mnist_train, mnist_test = mnist_ds['train'], mnist_ds['test']

#let's make vaidation dataset from train by taking 10%of it

num_valid_samples = 0.1 * mnist_info.splits['train'].num_examples
# to make sure all validation data points are integers

num_valid_samples = tf.cast(num_valid_samples, dtype = tf.int64)

#same process to make a dedicated variable for test data

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, dtype = tf.int64)


**Introduce a Scaling function**

In [ ]:
def scale(image, label):
  image = tf.cast(image, dtype = tf.float32)
  image /= 255.  #to scale all data point between 0 and 1
  return image, label

In [ ]:
scaled_train_and_validation_data = mnist_train.map(sacle)
scaled_test_data = mnist_test.map(scale)

**Shuffling and Batching**
Shuffuling is to keeping the same information but in a random order.
Batching is to use the computational power efficiently, used when we have to deal with enormous datasets and can not shuffule the data at once

**In Shuffling** if buffer_size = 1: No shuffling will takes place , if buffer_size >= Number of samples: Shuffling will happen only once , 
if buffer_size < Number of samples: we will be optimizing computational power


In [ ]:
buffer_size = 10000  # num of samples we are taking at a time to suffle
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(buffer_size)
test_data = scaled_test_data.shuffle(buffer_size)

#extracting validation and training dataset

validation_data = shuffled_train_and_validation_data.take(num_valid_samples)
train_data = shuffled_train_and_validation_data.skip(num_valid_samples)


**Batching** 1 < batch_size < number of samples:    Mini batch Gradient Descent (GD)

In [ ]:
#Batching

batch_size = 100
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_valid_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

**Outlining Model**

In [ ]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([ tf.keras.layers.Flatten(input_shape = (28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')])

#selecting loss and optimizer

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')

**Training**

In [ ]:
epochs_num = 5
model.fit(train_data, epochs = epochs_num, validation_data=(validation_inputs,validation_targets), verbose= 2)

Epoch 1/5
540/540 - 9s - loss: 0.2648 - accuracy: 0.9216 - val_loss: 0.1301 - val_accuracy: 0.9592
Epoch 2/5
540/540 - 9s - loss: 0.1016 - accuracy: 0.9686 - val_loss: 0.0760 - val_accuracy: 0.9757
Epoch 3/5
540/540 - 9s - loss: 0.0682 - accuracy: 0.9784 - val_loss: 0.0690 - val_accuracy: 0.9785
Epoch 4/5
540/540 - 9s - loss: 0.0538 - accuracy: 0.9829 - val_loss: 0.0532 - val_accuracy: 0.9833
Epoch 5/5
540/540 - 9s - loss: 0.0439 - accuracy: 0.9855 - val_loss: 0.0404 - val_accuracy: 0.9863


**Let's Test the accuracy of the model**

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 7ms/step - loss: 0.0706 - accuracy: 0.9779


In [ ]:
print('test_loss: {0:.2f}, test_accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100))

test_loss: 0.07, test_accuracy: 97.79%
